Download the Titanic dataset <a href="https://pythonprogramming.net/static/downloads/machine-learning-data/titanic.xls">here</a>  

This is what each column means:
- **Pclass**: Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
- **survival**: Survival (0 = No; 1 = Yes)
- **name**: Name
- **sex**: Sex
- **age**: Age
- **sibsp**: Number of Siblings/Spouses Aboard
- **parch**: Number of Parents/Children Aboard
- **ticket**: Ticket Number
- **fare**: Passenger Fare (British pound)
- **cabin**: Cabin
- **embarked**: Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
- **boat**: Lifeboat
- **body**: Body Identification Number
- **home**:.dest Home/Destination

The main focus on this dataset is typically on the survival column. When using supervised machine learning, we might train the data against the survival column as the classification. With clustering, however, we let the machine make the groups, and basically a label of its own.  
Our first interest is if the groups are clearly related to any of the columns, especially the survival column. For this data, we'll do flat-clustering, which is where we tell the machine we want two groups, but later we'll also let the machine determine the number of groups.

Lets start with imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn import preprocessing, model_selection

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

## Reading the data
You may need to do `pip install xlrd` before reading the excel data.

In [2]:
df = pd.read_excel('data/titanic.xls')
# df.head()

## Cleaning data
First, lets drop the useless columns `body` and `name`

In [3]:
df.drop(['body', 'name'], 1, inplace=True)
df.fillna(0, inplace=True)

The issue is, we've got non-numerical data here. The machine learning algorithm is going to require numbers. We can just drop the name column, it has no use to us.  
There are many ways to handle for non-numerical data.  
First, we will cycle through the columns in the Pandas dataframe. For columns that are not numbers, we will find their unique elements. This can be done by simply take a set of the column values. From here, the index within that set can be the new "numerical" value or "id" of the text data.  

- The embedded function `convert_to_int` converts the parameter value to whatever the value of that item (as a key) is from the text_digit_vals dictionary.
- If the value in not an `int` or a `float` we will convert the column to a list of its values, then we take the set of that column to get just the unique values.
- For each of the unique elements we find, we create a new dictionary key that is that unique element, with a value of a new number. 
- Once we've iterated through all of the unique values in the column, we then use mapping to map the function we created before to the pandas column. 

In [4]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

In [5]:
df = handle_non_numerical_data(df)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,755,211.3375,155,1,2,19
1,1,1,0,0.9167,1,2,519,151.5500,136,1,25,367
2,1,0,1,2.0000,1,2,519,151.5500,136,1,0,367
3,1,0,0,30.0000,1,2,519,151.5500,136,1,0,367
4,1,0,1,25.0000,1,2,519,151.5500,136,1,0,367


## Training
Lets drop the `survived` column, seperate the data against labels, scale the data and train the classifier

In [16]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

clf = KMeans(n_clusters=2)
clf.fit(X)

KMeans(n_clusters=2)

One important point to note here is that `survived` is either a 0, which means non-survival, or a 1, which means survival.  
For a clustering algorithm, **the machine will find the clusters, but then will asign arbitrary values to them, in the order it finds them**. Thus, the group that is survivors might be a 0 or a 1, depending on a degree of randomness. Thus, if you consistently get 30% and 70% accuracy, then your model is 70% accurate.  
Let's see what we get:

In [17]:
correct = 0
for i in range(len(X)):
    predict_row = np.array(X[i].astype(float))
    predict_row = predict_row.reshape(-1, len(predict_row))
    prediction = clf.predict(predict_row)
    
    if prediction[0] == y[i]:
        correct+=1

print("Accuracy: ", correct/len(X))

Accuracy:  0.2696715049656226


Note that the scaling step (while fitting the classifier) makes a big impact. If you remove that step, accuracy drops to 50%

We can see that this clustering algorithm seems to automatically categorize these people into who might survive or not on the ship's sinking.  
We don't have much in the way of determining exactly what the machine is thinking about and why these are the groups chosen, but they appear to have a high degree of correlation with survivability.

For testing, you can simply drop columns like `sex`, `boat` etc. one by one and try to see the impact on our accuracy.

For getting proper accuracy everytime (instead of alternating between 70 and 30 or whatever numbers) you can add this simple logic

In [18]:
print("Accuracy: ", max(correct/len(X), 1-(correct/len(X))))

Accuracy:  0.7303284950343774
